In [4]:
import numpy as np
import os
import tensorflow as tf

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ######

# one_hot encoding 0, 1, 2, 3, 4 for all labels
def one_hot_encoding(y):
    tmp_y = np.zeros([y.shape[0], 5])
    for i in range(y.shape[0]):
        tmp_y[i][y[i]] = 1
    return tmp_y

# split dataset into training set and one validation set
def split_folds(indices, Inputs, Labels, cross_validation, fold):
    n = Inputs.shape[0]
    if fold == cross_validation:
        validation_size = n - (int(n/cross_validation) * (cross_validation-1))
        X_train_idx, X_validate_idx = indices[:(n-validation_size)], indices[(n-validation_size):]
        y_train_idx, y_validate_idx = indices[:(n-validation_size)], indices[(n-validation_size):]
    else:
        validation_size = int(n/cross_validation)
        X_train_idx, X_validate_idx = np.concatenate((indices[:validation_size*(fold-1)], indices[validation_size*fold:]), axis=0), indices[(validation_size*(fold-1)):(validation_size*fold)]
        y_train_idx, y_validate_idx = np.concatenate((indices[:validation_size*(fold-1)], indices[validation_size*fold:]), axis=0), indices[(validation_size*(fold-1)):(validation_size*fold)]
    X_train, X_validate = np.array(Inputs[X_train_idx,:]), np.array(Inputs[X_validate_idx,:])
    y_train, y_validate = np.array(Labels[y_train_idx,:]), np.array(Labels[y_validate_idx,:])
    return X_train, y_train, X_validate, y_validate

def train(X_train, y_train, X_validate, y_validate, optimizer, epoch_bound, stop_threshold, batch_size, testing=False):

    global saver
    
    early_stop = 0
    winner_accuracy = 0.0
    winner_loss = np.infty
    
    for epoch in range(epoch_bound):

        # randomize training set
        indices_training = np.random.permutation(X_train.shape[0])
        X_train, y_train = X_train[indices_training,:], y_train[indices_training,:]

        # split training set into multiple mini-batches and start training
        total_batches = int(X_train.shape[0] / batch_size)
        for batch in range(total_batches):
            if batch == total_batches - 1:
                sess.run(optimizer, feed_dict={x: X_train[batch*batch_size:], y: y_train[batch*batch_size:]})
            else:
                sess.run(optimizer, feed_dict={x: X_train[batch*batch_size : (batch+1)*batch_size], y: y_train[batch*batch_size : (batch+1)*batch_size]})

        # compute validation accuracy
        cur_accuracy, cur_loss = evaluate(X_validate, y_validate)

        # If the accuracy rate does not increase for many times, it will early stop epochs-loop 
        if winner_accuracy < cur_accuracy:
            early_stop = 0
            winner_accuracy = cur_accuracy
            winner_loss = cur_loss
            # save best model in testing phase
            if testing == True:
                save_path = saver.save(sess, "./saved_model/Team35_HW2.ckpt")
        else:
            early_stop += 1
        if early_stop == stop_threshold:
            break

    return winner_accuracy, winner_loss

# evaluate model: compute accuracy, precision, recall
def evaluate(Inputs, Labels):
    global accuracy, cross_entropy_loss
    acc = sess.run(accuracy, feed_dict={x: Inputs, y:Labels})
    loss = sess.run(cross_entropy_loss, feed_dict={x: Inputs, y:Labels})
        
    return acc, loss

########## Data Processing ##########
# one_hot encoding 0, 1, 2, 3, 4 for all labels
y_train1 = one_hot_encoding(y_train1)
y_valid1 = one_hot_encoding(y_valid1)
y_test1 = one_hot_encoding(y_test1)

Inputs = np.concatenate((X_train1, X_valid1), axis=0)
Labels = np.concatenate((y_train1, y_valid1), axis=0)
########## Data Processing ##########

########## Define hyper-parameters ##########
learning_rate = 0.01
batch_size = 32
epoch_bound = 1000
cross_validation = 5
stop_threshold = 20
########## Define hyper-parameters ##########

########## Build DNN ##########
# define placeholder for inputs and labels
x = tf.placeholder(tf.float32, [None, Inputs.shape[1]], name="X")
y = tf.placeholder(tf.float32, [None, Labels.shape[1]], name="y")

# add 5 hidden layers with 128 neurals each
h1 = tf.layers.dense(inputs=x,  units=128, activation=tf.nn.elu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer(), name="dnn_h1")
h2 = tf.layers.dense(inputs=h1, units=128, activation=tf.nn.elu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer(), name="dnn_h2")
h3 = tf.layers.dense(inputs=h2, units=128, activation=tf.nn.elu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer(), name="dnn_h3")
h4 = tf.layers.dense(inputs=h3, units=128, activation=tf.nn.elu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer(), name="dnn_h4")
h5 = tf.layers.dense(inputs=h4, units=128, activation=tf.nn.elu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer(), name="dnn_h5")
# add outputs softmax layer with 5 neurals
logits = tf.layers.dense(inputs=h5, units=5, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), name="logits")
outputs = tf.nn.softmax(logits, name="Y_probability")

# cross entropy loss function
cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name="loss")

# accuracy
correct_prediction = tf.equal(tf.argmax(outputs, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

# training iteration and define Adam optimizer with learning rate
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss, name="training_op")
########## Build DNN ##########

########## Training ##########
print("########## Start training without dopout ##########")
# start TF session
sess = tf.Session()
init = tf.global_variables_initializer()

# randomize dataset
indices = np.random.permutation(Inputs.shape[0])

# start cross validation
avg_accuracy = 0.0

for fold in range(1, cross_validation+1):
    
    # init weights
    sess.run(init)
    
    # split inputs into training set and validation set for each fold
    X_train, y_train, X_validate, y_validate = split_folds(indices, Inputs, Labels, cross_validation, fold)
    
    winner_accuracy, winner_loss = train(X_train, y_train, X_validate, y_validate, optimizer, epoch_bound, stop_threshold, batch_size, testing=False)
    
    avg_accuracy += winner_accuracy
    print("########## Fold:", fold, "##########")
    print("Loss:", winner_loss)
    print("Accuracy:", winner_accuracy)

avg_accuracy /= cross_validation
print("Average accuracy of cross validation:", avg_accuracy)
sess.close()
########## Training ##########

########## Testing ##########
print("########## Testing results ##########")
# start TF session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# init saver to resrore the best model
saver = tf.train.Saver()

# randomize dataset
indices = np.random.permutation(Inputs.shape[0])
Inputs, Labels = np.array(Inputs[indices,:]), np.array(Labels[indices,:])

# get validation set with the size of a batch
X_train, y_train = Inputs[batch_size:], Labels[batch_size:]
X_validate, y_validate = Inputs[:batch_size], Labels[:batch_size]

# start training all the inputs
winner_accuracy, winner_loss = train(X_train, y_train, X_validate, y_validate, optimizer, epoch_bound, stop_threshold, batch_size, testing=True)

saver.restore(sess, "./saved_model/Team35_HW2.ckpt")
test_accuracy, test_loss = evaluate(X_test1, y_test1)

print("Loss:", test_loss)
print("Accuracy:", test_accuracy)
sess.close()
########## Test ##########

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
########## Start training without dopout ##########
########## Fold: 1 ##########
Loss: 0.110268
Accuracy: 0.978591
########## Fold: 2 ##########
Loss: 0.100484
Accuracy: 0.976303
########## Fold: 3 ##########
Loss: 0.145818
Accuracy: 0.959961
########## Fold: 4 ##########
Loss: 0.103289
Accuracy: 0.979245
########## Fold: 5 ##########
Loss: 0.110468
Accuracy: 0.978105
Average accuracy of cross validation: 0.974440979958
########## Testing results ##########
INFO:tensorflow:Restoring parameters from ./saved_model/Team35_HW2.ckpt
Loss: 0.102972
Accuracy: 0.984822
